In [33]:
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'   # Use GPU if available, otherwise CPU

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [34]:
# Define the neural network model
class MLP(nn.Module):
    def __init__(self, activation_fn):
        super(MLP, self).__init__()

        self.activation_fn = activation_fn

        # Define fully connected layers
        self.l1 = nn.Linear(784, 520)  # Input layer (28x28=784) to hidden layer
        self.l2 = nn.Linear(520, 320)  # Hidden layer
        self.l3 = nn.Linear(320, 240)  # Hidden layer
        self.l4 = nn.Linear(240, 120)  # Hidden layer
        self.l5 = nn.Linear(120, 10)   # Output layer (10 classes)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.activation_fn(self.l1(x))
        x = self.activation_fn(self.l2(x))
        x = self.activation_fn(self.l3(x))
        x = self.activation_fn(self.l4(x))
        return self.l5(x)

In [35]:
# Save the model with different activation functions
model_relu = MLP(F.relu).to(device)
model_sig = MLP(F.sigmoid).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

optimizer_relu = torch.optim.Adam(model_relu.parameters(),  
                             lr=0.001, 
                             betas=(0.9, 0.999),
                             eps=1e-08,
                             weight_decay=0,
                             amsgrad=False)

optimizer_sig = torch.optim.Adam(model_sig.parameters(),   
                             lr=0.001, 
                             betas=(0.9, 0.999),
                             eps=1e-08,
                             weight_decay=0,
                             amsgrad=False)


# Training function
def train(epoch, model, optimizer, activation):
    model.train()   # Set the model to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move data and target to the device
        data, target = data.to(device), target.to(device)   
        optimizer.zero_grad()               # Zero the gradients
        output = model(data)                # Forward pass
        loss = criterion(output, target)    # Compute loss
        loss.backward()                     # Backward pass
        optimizer.step()                    # Update weights
        
        if batch_idx % 10 == 0:
            print('[{}] Train Epoch: {} | Batch: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                activation,
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))

In [36]:
# Testing function
def test(model, activation):
    model.eval()    # Set model to evaluation mode
    test_loss = 0   # Initialize test loss
    correct = 0     # Initialize correct predictions counter

    with torch.no_grad():  # Disable gradient calculation
        for data, target in test_loader:
            # Move data and target to the device
            data, target = data.to(device), target.to(device)
            output = model(data)                            # Forward pass
            test_loss += criterion(output, target).item()   # Accumulate loss

            # Get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]      
            # Count correct predictions
            correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    # Average the test loss
    test_loss /= len(test_loader.dataset)

    print(f'\n[{activation}] Test set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.2f}%)\n')

In [ ]:
# Main function
if __name__ == '__main__':
    since = time.time()

    for epoch in range(1, 10):
        print(f"\n====== Epoch {epoch} ======")

        # ReLU Model Test & Evaluation
        epoch_start = time.time()
        train(epoch, model_relu, optimizer_relu, 'ReLU')
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'[ReLU] Training time: {m:.0f}m {s:.0f}s')

        test(model_relu, 'ReLU')
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'[ReLU] Testing time: {m:.0f}m {s:.0f}s')

    for epoch in range(1, 10):
        print(f"\n====== Epoch {epoch} ======")
        
        # Sigmoid Model Test & Evaluation
        epoch_start = time.time()
        train(epoch, model_sig, optimizer_sig, 'Sigmoid')
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'[Sigmoid] Training time: {m:.0f}m {s:.0f}s')

        test(model_sig, 'Sigmoid')
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'[Sigmoid] Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'\nTotal Time: {m:.0f}m {s:.0f}s')
    print(f'Models were trained on {device}!')


====== Epoch 1 ======
[ReLU] Train Epoch: 1 | Batch: 0/60000 (0%) | Loss: 2.309318
[ReLU] Train Epoch: 1 | Batch: 640/60000 (1%) | Loss: 2.030342
[ReLU] Train Epoch: 1 | Batch: 1280/60000 (2%) | Loss: 1.341074
[ReLU] Train Epoch: 1 | Batch: 1920/60000 (3%) | Loss: 1.065203
[ReLU] Train Epoch: 1 | Batch: 2560/60000 (4%) | Loss: 0.504554
[ReLU] Train Epoch: 1 | Batch: 3200/60000 (5%) | Loss: 0.377749
[ReLU] Train Epoch: 1 | Batch: 3840/60000 (6%) | Loss: 0.737845
[ReLU] Train Epoch: 1 | Batch: 4480/60000 (7%) | Loss: 0.709373
[ReLU] Train Epoch: 1 | Batch: 5120/60000 (9%) | Loss: 0.501425
[ReLU] Train Epoch: 1 | Batch: 5760/60000 (10%) | Loss: 0.321456
[ReLU] Train Epoch: 1 | Batch: 6400/60000 (11%) | Loss: 0.674172
[ReLU] Train Epoch: 1 | Batch: 7040/60000 (12%) | Loss: 0.320585
[ReLU] Train Epoch: 1 | Batch: 7680/60000 (13%) | Loss: 0.333628
[ReLU] Train Epoch: 1 | Batch: 8320/60000 (14%) | Loss: 0.425669
[ReLU] Train Epoch: 1 | Batch: 8960/60000 (15%) | Loss: 0.394829
[ReLU] Train Ep